In [4]:
import sqlite3
from os import path
from time import time
from html2corpus import HTML2Corpus
from html2corpus.extractors import ReadabilityExtractor

In [5]:
site = 'Junge Freiheit'
labels = ['KW44', 'KW45', 'KW46', 'KW47', 'KW48', 'KW49']
files = [path.join('data', 'Corpus_{}_{}.txt'.format(label, site)) for label in labels]
dates = [('2015-10-26', '2015-11-02', site),
         ('2015-11-02', '2015-11-09', site),
         ('2015-11-09', '2015-11-16', site),
         ('2015-11-16', '2015-11-23', site),
         ('2015-11-23', '2015-11-30', site),
         ('2015-11-30', '2015-12-07', site)]

db_file = path.join('..', 'Crawler', 'data', 'de_news.sqlite')
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

In [6]:
def get_data(kw):
    query = '''SELECT title, html 
               FROM Articles 
               LEFT JOIN Sources ON Articles.source = Sources.id 
               WHERE date BETWEEN ? and ?
               AND site = ?'''
    cursor.execute(query, kw)

    data = cursor.fetchmany(100)
    
    while len(data) > 0:
        for row in data:
            title, html = row
            yield html
        data = cursor.fetchmany(100)

for file, date in zip(files, dates):
    HTML2Corpus(get_data(date), extractor=ReadabilityExtractor(min_len=100)).save(file)


conn.close()